In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_gbq import read_gbq

# --- Configuration ---
# Display settings to see everything in the text report
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:.1%}'.format
cdr = os.environ.get("WORKSPACE_CDR", "")

print("--- CONTRAST FREQUENCY & DISCOVERY ANALYSIS ---")

# ==============================================================================
# 1. SQL QUERY
# ==============================================================================
sql_analysis = f"""
WITH ContrastSignals AS (
    -- 1. Drugs: Look for 'Contrast Media' class or specific keywords
    SELECT DISTINCT person_id, drug_exposure_start_date AS event_date
    FROM `{cdr}.drug_exposure` d
    JOIN `{cdr}.concept` c ON d.drug_concept_id = c.concept_id
    WHERE c.concept_class_id = 'Contrast Media'
       OR REGEXP_CONTAINS(c.concept_name, r'(?i)(Iohexol|Iopamidol|Gadolinium|Iodixanol|Omnipaque|Visipaque|Optiray)')

    UNION ALL

    -- 2. Procedures: Explicit 'With Contrast' codes
    SELECT DISTINCT person_id, procedure_date AS event_date
    FROM `{cdr}.procedure_occurrence` p
    JOIN `{cdr}.concept` c ON p.procedure_concept_id = c.concept_id
    WHERE REGEXP_CONTAINS(c.concept_name, r'(?i)with contrast|w/ contrast|w contrast')
      AND NOT REGEXP_CONTAINS(c.concept_name, r'(?i)without contrast')
),

DistinctContrastEvents AS (
    SELECT DISTINCT person_id, event_date FROM ContrastSignals
),

AllProcedures AS (
    SELECT
        p.person_id,
        p.procedure_date,
        p.procedure_concept_id,
        c.concept_name,
        c.concept_code
    FROM `{cdr}.procedure_occurrence` p
    JOIN `{cdr}.concept` c ON p.procedure_concept_id = c.concept_id
    WHERE 
        c.domain_id IN ('Procedure', 'Measurement')
        AND (
            c.concept_class_id IN ('Radiology', 'Procedure', 'CT Scan', 'MRI', 'NM', 'Ultrasound')
            OR c.vocabulary_id IN ('CPT4', 'HCPCS', 'ICD10PCS')
        )
)

SELECT
    proc.procedure_concept_id,
    proc.concept_name,
    COUNT(*) AS total_n,
    COUNTIF(c.person_id IS NOT NULL) AS with_contrast_n,
    SAFE_DIVIDE(COUNTIF(c.person_id IS NOT NULL), COUNT(*)) AS freq
FROM AllProcedures proc
LEFT JOIN DistinctContrastEvents c
    ON proc.person_id = c.person_id 
    AND proc.procedure_date = c.event_date
GROUP BY 1, 2
HAVING total_n > 50 
ORDER BY freq DESC
"""

print("Running Query... (Approx 1-2 mins)")
df = read_gbq(sql_analysis, dialect="standard")

# ==============================================================================
# 2. CLASSIFICATION LOGIC
# ==============================================================================

def guess_modality(name):
    n = name.lower()
    if 'mri' in n or 'magnetic resonance' in n: return 'MRI'
    if 'ct' in n or 'computed tom' in n: return 'CT'
    if 'angiography' in n: return 'Angio'
    if 'x-ray' in n or 'radiologic' in n: return 'X-Ray'
    if 'ultrasound' in n or 'sonography' in n: return 'Ultrasound'
    if 'fluoroscopy' in n: return 'Fluoro'
    if 'mammography' in n: return 'Mammo'
    return 'Other'

def analyze_name_claims(name):
    n = name.lower()
    has_with = 'with contrast' in n or 'w/ contrast' in n
    has_without = 'without contrast' in n or 'w/o contrast' in n
    
    if has_with and has_without: return 'Both'
    if has_with: return 'Explicit "With"'
    if has_without: return 'Explicit "Without"'
    return 'Unspecified'

df['Modality'] = df['concept_name'].apply(guess_modality)
df['Name_Type'] = df['concept_name'].apply(analyze_name_claims)

# Define Categories
conditions = [
    (df['freq'] >= 0.90),
    (df['freq'] >= 0.10) & (df['freq'] < 0.90),
    (df['freq'] < 0.10)
]
choices = ['High (Protocol)', 'Mixed (Ambiguous)', 'Low (Coincidental)']
df['Category'] = np.select(conditions, choices, default='Low')

# ==============================================================================
# 3. TEXT REPORT (Detailed Lists)
# ==============================================================================

def print_category_full(category_name, modality_filter=None):
    subset = df[df['Category'] == category_name].copy()
    
    if modality_filter:
        subset = subset[subset['Modality'].isin(modality_filter)]
    
    # Sort: Modality first, then Volume
    subset = subset.sort_values(['Modality', 'total_n'], ascending=[True, False])
    
    print(f"\n{'#'*120}")
    print(f"CATEGORY: {category_name.upper()}")
    if modality_filter: print(f"(Filtered Modalities: {modality_filter})")
    print(f"Total Procedures: {len(subset)}")
    print(f"{'#'*120}\n")
    
    # Header
    print(f"{'Procedure Name':<60} | {'Type':<6} | {'Name Says':<16} | {'Total':>8} | {'w/Contr':>8} | {'w/o Contr':>9} | {'% w/Con':>7}")
    print("-" * 130)
    
    for _, row in subset.iterrows():
        # Truncate long names for readability
        name = (row['concept_name'][:57] + '...') if len(row['concept_name']) > 57 else row['concept_name']
        without_n = row['total_n'] - row['with_contrast_n']
        
        # Flag suspicious rows
        flag = " (!)" if row['Name_Type'] == 'Explicit "Without"' else ""
        
        print(f"{name:<60} | {row['Modality']:<6} | {row['Name_Type'] + flag:<16} | {row['total_n']:8,} | {row['with_contrast_n']:8,} | {without_n:9,} | {row['freq']:7.1%}")

# --- Execute Print ---
print_category_full('High (Protocol)')
print_category_full('Mixed (Ambiguous)', modality_filter=['CT', 'MRI', 'Angio', 'X-Ray', 'Ultrasound', 'Fluoro', 'Mammo'])

# ==============================================================================
# 4. VISUALIZATION
# ==============================================================================
print("\nGeneratng Visualization...")

plt.figure(figsize=(14, 9))
sns.scatterplot(
    data=df, 
    x='total_n', 
    y='freq', 
    hue='Category',
    palette={'High (Protocol)': 'red', 'Mixed (Ambiguous)': 'orange', 'Low (Coincidental)': 'gray'},
    alpha=0.65,
    size='total_n',
    sizes=(20, 500)
)

# Plot Styling
plt.xscale('log')
plt.title("Procedure Map: Volume vs. Contrast Frequency", fontsize=16)
plt.xlabel("Total Procedure Volume (Log Scale)", fontsize=12)
plt.ylabel("Frequency of Contrast Co-occurrence (0-1)", fontsize=12)

# Threshold Lines
plt.axhline(0.9, color='red', linestyle='--', alpha=0.4, label='90% Protocol Threshold')
plt.axhline(0.1, color='gray', linestyle='--', alpha=0.4, label='10% Noise Threshold')

# Legend
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', frameon=True)
plt.grid(True, which="major", linestyle='-', alpha=0.2)
plt.grid(True, which="minor", linestyle=':', alpha=0.1)

plt.tight_layout()
plt.show()

# ==============================================================================
# 5. EXPORT
# ==============================================================================
df.to_csv("contrast_procedure_frequency_full.csv", index=False)
print("Analysis Complete. Data saved to 'contrast_procedure_frequency_full.csv'")

In [ ]:
import os
import pandas as pd
import numpy as np
from pandas_gbq import read_gbq

# --- Configuration ---
pd.set_option('display.max_rows', None)  # Allow printing ALL rows
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.1%}'.format
cdr = os.environ.get("WORKSPACE_CDR", "")

print("--- CONTRAST FREQUENCY & NAME ANALYSIS ---")

# ==============================================================================
# 1. SQL QUERY (Same logic, identifying co-occurrences)
# ==============================================================================
sql_analysis = f"""
WITH ContrastSignals AS (
    -- 1. Drugs
    SELECT DISTINCT person_id, drug_exposure_start_date AS event_date
    FROM `{cdr}.drug_exposure` d
    JOIN `{cdr}.concept` c ON d.drug_concept_id = c.concept_id
    WHERE c.concept_class_id = 'Contrast Media'
       OR REGEXP_CONTAINS(c.concept_name, r'(?i)(Iohexol|Iopamidol|Gadolinium|Iodixanol|Omnipaque|Visipaque|Optiray)')

    UNION ALL

    -- 2. Procedures (Explicit 'With Contrast')
    SELECT DISTINCT person_id, procedure_date AS event_date
    FROM `{cdr}.procedure_occurrence` p
    JOIN `{cdr}.concept` c ON p.procedure_concept_id = c.concept_id
    WHERE REGEXP_CONTAINS(c.concept_name, r'(?i)with contrast|w/ contrast|w contrast')
      AND NOT REGEXP_CONTAINS(c.concept_name, r'(?i)without contrast')
),

DistinctContrastEvents AS (
    SELECT DISTINCT person_id, event_date FROM ContrastSignals
),

AllProcedures AS (
    SELECT
        p.person_id,
        p.procedure_date,
        p.procedure_concept_id,
        c.concept_name,
        c.concept_code
    FROM `{cdr}.procedure_occurrence` p
    JOIN `{cdr}.concept` c ON p.procedure_concept_id = c.concept_id
    WHERE 
        c.domain_id IN ('Procedure', 'Measurement')
        AND (
            c.concept_class_id IN ('Radiology', 'Procedure', 'CT Scan', 'MRI', 'NM', 'Ultrasound')
            OR c.vocabulary_id IN ('CPT4', 'HCPCS', 'ICD10PCS')
        )
)

SELECT
    proc.procedure_concept_id,
    proc.concept_name,
    COUNT(*) AS total_n,
    COUNTIF(c.person_id IS NOT NULL) AS with_contrast_n,
    SAFE_DIVIDE(COUNTIF(c.person_id IS NOT NULL), COUNT(*)) AS freq
FROM AllProcedures proc
LEFT JOIN DistinctContrastEvents c
    ON proc.person_id = c.person_id 
    AND proc.procedure_date = c.event_date
GROUP BY 1, 2
HAVING total_n > 50 
ORDER BY freq DESC
"""

print("Running Query... (Approx 1-2 mins)")
df = read_gbq(sql_analysis, dialect="standard")

# ==============================================================================
# 2. PYTHON CLASSIFICATION
# ==============================================================================

# A. Modality Guesser
def guess_modality(name):
    n = name.lower()
    if 'mri' in n or 'magnetic resonance' in n: return 'MRI'
    if 'ct' in n or 'computed tom' in n: return 'CT'
    if 'x-ray' in n or 'radiologic' in n: return 'X-Ray'
    if 'ultrasound' in n or 'sonography' in n: return 'Ultrasound'
    if 'angiography' in n: return 'Angio'
    return 'Other'

df['Modality'] = df['concept_name'].apply(guess_modality)

# B. Name Text Analysis (The "What does it CLAIM to be?" check)
def analyze_name_claims(name):
    n = name.lower()
    has_with = 'with contrast' in n or 'w/ contrast' in n
    has_without = 'without contrast' in n or 'w/o contrast' in n
    
    if has_with and has_without:
        return 'Both (w/ & w/o)' # Usually biphasic scans
    elif has_with:
        return 'Explicit "With"'
    elif has_without:
        return 'Explicit "Without"'
    else:
        return 'Neither/Unspecified'

df['Name_Type'] = df['concept_name'].apply(analyze_name_claims)

# C. Frequency Categorization (The "What IS it actually?" check)
conditions = [
    (df['freq'] >= 0.90),
    (df['freq'] >= 0.10) & (df['freq'] < 0.90), # Lowered threshold to 10% to catch more mixed
    (df['freq'] < 0.10)
]
choices = ['High (Protocol)', 'Mixed (Ambiguous)', 'Low (Coincidental)']
df['Category'] = np.select(conditions, choices, default='Low')

# ==============================================================================
# 3. OUTPUT GENERATION
# ==============================================================================

def print_full_category(category_name, modality_filter=None):
    subset = df[df['Category'] == category_name].copy()
    
    if modality_filter:
        subset = subset[subset['Modality'].isin(modality_filter)]
    
    # Sort: Put the biggest contrast users at the top
    subset = subset.sort_values(['Modality', 'total_n'], ascending=[True, False])
    
    print(f"\n{'='*120}")
    print(f"CATEGORY: {category_name.upper()}")
    if modality_filter: print(f"(Filtered to: {modality_filter})")
    print(f"Count: {len(subset)} procedures")
    print(f"{'='*120}")
    
    # Header
    # Name | Modality | Name Claims | Total | w/ Contrast | % Freq
    header = f"{'Procedure Name':<55} | {'Modality':<6} | {'Name Says':<16} | {'Total':>7} | {'w/Contr':>7} | {'Freq':>6}"
    print(header)
    print("-" * 120)
    
    for _, row in subset.iterrows():
        # Truncate name for display
        name = (row['concept_name'][:52] + '...') if len(row['concept_name']) > 52 else row['concept_name']
        
        # Highlight interesting mismatches with a *
        # e.g., Name says "Without" but frequency is > 10%
        attention_flag = ""
        if row['Name_Type'] == 'Explicit "Without"':
            attention_flag = " (!)" 
        
        print(f"{name:<55} | {row['Modality']:<6} | {row['Name_Type'] + attention_flag:<16} | {row['total_n']:7,} | {row['with_contrast_n']:7,} | {row['freq']:6.1%}")

# --- EXECUTION ---

# 1. PRINT ALL HIGH FREQUENCY
# These are your core "Contrast" cohort candidates
print_full_category('High (Protocol)')

# 2. PRINT ALL MIXED FREQUENCY (Imaging Only)
# These are the tricky ones. Look for 'Explicit "Without" (!)' tags here.
# Identifying these helps you decide if you need to be stricter with your 'Without Contrast' cohort definitions.
print_full_category('Mixed (Ambiguous)', modality_filter=['CT', 'MRI', 'Angio', 'X-Ray', 'Ultrasound'])

# 3. Summary Stats of Name Types
print(f"\n{'='*60}")
print("SUMMARY: Name Syntax vs Actual Usage")
print(f"{'='*60}")
print(df.groupby(['Category', 'Name_Type']).size().unstack(fill_value=0))